In [0]:
# save image as grayscale

import cv2
image = cv2.imread("lena.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imwrite("grayscale.jpg", image)

True

In [0]:
# DCT

In [0]:
# DCT matrix
import numpy as np

def DCT_matrix(N):    
  W = np.zeros((N,N))
  for m in range (N):
    for n in range (N):
      if m == 0:
        W[m,n] = np.sqrt(2/N) * 1/np.sqrt(2)
      else:
        W[m,n] = np.sqrt(2/N) * np.cos((2 * n + 1) * (m * np.pi) / (2 * N))
  return W

In [0]:
# function to save matrix to a textfile
import numpy 

def save_matrix(name, matrix):
  numpy.savetxt(name, matrix, delimiter = ',')  

In [0]:
# initialize DCT and transpose DCT matrices, save DCT matrix to text file
import numpy as np

dctmtx8 = DCT_matrix(8)
dctmtx8t = np.transpose(dctmtx8)
save_matrix("dctmtx.txt", dctmtx8)

In [0]:
# perform matrix multiplication to obtain DCT coefficients

import numpy as np
import cv2

image = cv2.imread("D15.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
transform = np.zeros(image.shape)
dctcoeff = np.zeros(image.shape)
im_h, im_w = image.shape[:2]
bl_h, bl_w = 8, 8

for row in np.arange(im_h - bl_h + 1, step=bl_h):
      for col in np.arange(im_w - bl_w + 1, step=bl_w):
        dctcoeff[row:row+bl_h, col:col+bl_w] = dctmtx8.dot(image[row:row+bl_h, col:col+bl_w]).dot(dctmtx8t)


In [0]:
# save DCT coefficient image and coefficients in a text file
cv2.imwrite("coefficient.jpg", dctcoeff)
save_matrix("dctcoeff.txt", dctcoeff)

In [0]:
# function to take upper left triangular region values (half values)

def half_matrix(a):  
  b = np.zeros(a.shape)
  for i in range (0, a.shape[0]):   
    for j in range (0, a.shape[1]):
      b[i:a.shape[0]-i,i] = a[i:a.shape[0]-i,i] 
      b[j,j:a.shape[0]-j] = a[j,j:a.shape[0]-j] 
      j += 1
  return b

In [0]:
# perform inverse DCT on new DCT coefficients, where only upper left values of DCT coefficient were used

inverse = np.zeros(image.shape)

for row in np.arange(im_h - bl_h + 1, step=bl_h):
    for col in np.arange(im_w - bl_w + 1, step=bl_w):        
      inverse[row:row+bl_h, col:col+bl_w] = dctmtx8t.dot(half_matrix(dctcoeff[row:row+bl_h, col:col+bl_w])).dot(dctmtx8)

cv2.imwrite("D15-dct.jpg", inverse)

True

In [0]:
# KLT

In [0]:
# function to get matrix covariance
import numpy as np

def covar(X):
  meanX = np.mean(X, axis = 0)
  lenX = X.shape[0]
  X = X - meanX
  covariance = np.transpose(X).dot(X)/lenX
  return covariance

In [0]:
# function to convert from PIL image to OpenCV for saving purposes

def pil2cv(image):
    ''' PIL型 -> OpenCV型 '''
    new_image = np.array(image, dtype=np.uint8)
    if new_image.ndim == 2: 
        pass
    elif new_image.shape[2] == 3:
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
    elif new_image.shape[2] == 4:
        new_image = cv2.cvtColor(new_image, cv2.COLOR_RGBA2BGRA)
    return new_image

In [0]:
from numpy import size
from scipy.sparse.linalg import eigs
import numpy as np

def eigen(arr):  
  # subtract the mean (along columns)
  M = (arr - np.mean(np.transpose(arr),axis=1)).T 
  # get largets 32 eigenvalues and eigenvectors of covariance matrix
  [eigenval, eigenvec] = eigs(covar(M), k=32, which='LM')  
  # sort eigenvalues based on index
  idx = eigenval.argsort() 
  idx = idx[::-1]
  # sorting eigenvectors according to the sorted eigenvalues
  eigenvec = eigenvec[:,idx]
  # sort eigenvalues in ascending order
  eigenval = eigenval[idx]   
  score = np.transpose(eigenvec).dot(M)
  return eigenvec, score, eigenval

In [0]:
import cv2
import numpy as np
from scipy.sparse.linalg import eigs

# load image
image = cv2.imread('D15.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
im_h, im_w = image.shape[:2]
bl_h, bl_w = 8, 8

# break image into blocks of 64x64
for row in np.arange(im_h - bl_h + 1, step=bl_h):
      for col in np.arange(im_w - bl_w + 1, step=bl_w):
        eigenvec, score, eigenval = eigen(np.array(image))
        klt = np.transpose(eigenvec.dot(score)) + np.mean(image, axis=0)  

cv2.imwrite("D15-klt.jpg", pil2cv(klt))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  after removing the cwd from sys.path.


True

In [0]:
# SVD

In [45]:
from numpy import array
from scipy.linalg import svd
import numpy as np
from scipy.sparse.linalg import svds
import cv2

image = cv2.imread("D15.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
svdimg = np.zeros(image.shape)
im_h, im_w = image.shape[:2]
bl_h, bl_w = 8, 8

# separate image into blocks of 8x8
for row in np.arange(im_h - bl_h + 1, step=bl_h):
      for col in np.arange(im_w - bl_w + 1, step=bl_w):
        U, ggt, VT = svd(image)
        ggt = np.diag(ggt)
        svdimg = U.dot(ggt).dot(VT)
        
cv2.imwrite("D15-svd.jpg", svdimg)

True

In [0]:
# function to obtain dominant N/2 values of a matrix

def max_values(ary, n):    
    maxhalf = np.zeros(ary.shape)
    maxhalf = maxhalf.flatten()
    flat = ary.flatten()
    indices = np.argpartition(flat, -n)[-n:]
    indices = indices[np.argsort(-flat[indices])]
    indices = flat[indices]
    for i in range (0, n):
      maxhalf[0:n] = indices[0:n]    
    maxhalf.shape = ary.shape
    return maxhalf

In [47]:
from numpy import array
from scipy.linalg import svd
import numpy as np
from scipy.sparse.linalg import svds

# define a matrix
image = cv2.imread("D15.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
svd_approx = np.zeros(image.shape)
im_h, im_w = image.shape[:2]
bl_h, bl_w = 8, 8

for row in np.arange(im_h - bl_h + 1, step=bl_h):
      for col in np.arange(im_w - bl_w + 1, step=bl_w):             
        svd_approx[row:row+bl_h, col:col+bl_w] = max_values(U[row:row+bl_h, col:col+bl_w], 4).dot(ggt[row:row+bl_h, col:col+bl_w]).dot(max_values(VT[row:row+bl_h, col:col+bl_w], 4))
        
cv2.imwrite("D15-svd-approx.jpg", svd_approx)

True